In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
# Loading the trained model, scaler pickle, and the label encoder

model = load_model('artifacts/ann_model.h5')

with open('artifacts/label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)


with open('artifacts/onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)


with open('artifacts/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [5]:
# Example input data

input_data = {
    'CreditScore': 650,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 45,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [ ]:
# Converting the 'Geography' column to one-hot encoding (Categorical to numerical)

geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

f:\pythoncode\ML\ML Projects\Churn-Modelling-using-ANN\venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [10]:
input_data_df = pd.DataFrame([input_data])
input_data_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,650,France,Male,45,3,60000,2,1,1,50000


In [11]:
# Combining the one-hot encoded 'Geography' column with the other input data

input_data_df = pd.concat([input_data_df, geo_encoded_df], axis=1)
input_data_df.drop(columns=['Geography'], inplace=True)
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,Male,45,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
# Converting the 'Gender' column to numerical using the label encoder

input_data_df['Gender'] = label_encoder_gender.transform(input_data_df['Gender'])
input_data_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,650,1,45,3,60000,2,1,1,50000,1.0,0.0,0.0


In [15]:
# Scaling the input data
input_scaled = scaler.transform(input_data_df)
input_scaled

array([[-0.01709861,  0.91324755,  0.58015577, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [16]:
# Predicting the probability of the customer leaving the bank
model_output = model.predict(input_scaled)
model_output

1/1 [==============================] - 0s 400ms/step


array([[0.05727741]], dtype=float32)

In [18]:
prediction_probability = model_output[0][0]
prediction_probability

0.057277415

In [19]:
if prediction_probability > 0.5:
    print('The customer is likely to leave the bank')
    print(f'Probability: {prediction_probability*100:.2f}%')
else:
    print('The customer is not likely to leave the bank')
    print(f'Probability: {prediction_probability*100:.2f}%')

The customer is not likely to leave the bank
Probability: 5.73%
